# Data Cleaning Notebook

In [1]:
import pandas as pd

Below I will look at my data and perform some cleaning operations to make the data more manageable. This will include dropping any duplicates, any null values, and converting the necessary columns to the right type.(e.g. date to datetime type)

**Below I will get familiar with my data by reading it in and looking at the data types and the columns.**

In [2]:
df = pd.read_csv('../data/playlist_tracks2.csv')
df.head()

,Track,Artist,Genre,Album Title,Album Type,Release Date,Thumbnail,Acousticness,Danceability,Energy,Instrumentalness,Popularity,Speechiness,Track ID
0,"Shook Ones, Pt. II",Mobb Deep,"east coast hip hop, hardcore hip hop, hip hop,...",The Infamous,album,1995-04-25,https://i.scdn.co/image/ab67616d00004851a2203f...,0.01460,0.763,0.786,0.011400,78,0.229,33ZXjLCpiINn8eQIDYEPTD
1,Hypnotize - 2014 Remaster,The Notorious B.I.G.,"east coast hip hop, gangster rap, hardcore hip...",Life After Death (2014 Remastered Edition),album,1997-03-04,https://i.scdn.co/image/ab67616d00004851fde79b...,0.14500,0.901,0.697,0.000000,83,0.280,7KwZNVEaqikRSBSpyhXK2j
2,Ambitionz Az A Ridah,2Pac,"g funk, gangster rap, hip hop, rap, west coast...",All Eyez On Me,album,1996-02-13,https://i.scdn.co/image/ab67616d00004851a786ad...,0.00253,0.801,0.908,0.000655,2,0.221,3ssX20QT5c3nA9wk78V1LQ
3,N.Y. State of Mind,Nas,"conscious hip hop, east coast hip hop, gangste...",Illmatic,album,1994-04-19,https://i.scdn.co/image/ab67616d00004851045fc9...,0.03940,0.665,0.910,0.000000,74,0.223,0trHOzAhNpGCsGBEu7dOJo
4,It Was A Good Day,Ice Cube,"conscious hip hop, g funk, gangster rap, hip h...",The Predator,album,1992-11-17,https://i.scdn.co/image/ab67616d00004851994c31...,0.33000,0.798,0.744,0.000106,84,0.136,2qOm7ukLyHUXWyR4ZWLwxA


Here we will look at the columns to decide how to approach the future cleaning of the data.  We can see that we have a Track ID column which will be useful for removing any duplicate data, the Release Date will be useful for creating a "Year" column which will be helpful in the modeling step, additionally the Release Data column needs to be cahnged to a datetime format.

In [3]:
df.columns

Index(['Track', 'Artist', 'Genre', 'Album Title', 'Album Type', 'Release Date',
       'Thumbnail', 'Acousticness', 'Danceability', 'Energy',
       'Instrumentalness', 'Popularity', 'Speechiness', 'Track ID'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13129 entries, 0 to 13128
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track             13129 non-null  object 
 1   Artist            13129 non-null  object 
 2   Genre             12316 non-null  object 
 3   Album Title       13129 non-null  object 
 4   Album Type        13129 non-null  object 
 5   Release Date      13129 non-null  object 
 6   Thumbnail         13129 non-null  object 
 7   Acousticness      13129 non-null  float64
 8   Danceability      13129 non-null  float64
 9   Energy            13129 non-null  float64
 10  Instrumentalness  13129 non-null  float64
 11  Popularity        13129 non-null  int64  
 12  Speechiness       13129 non-null  float64
 13  Track ID          13129 non-null  object 
dtypes: float64(5), int64(1), object(8)
memory usage: 1.4+ MB


---
Below I will check for any missing values as well as duplicate values. 

**NOTE: Since the data was pull from multiple playlists found in Spotify there will be a sum of duplicated values, I knew going into this that this would be an issue so I made sure to gather a variety or artists, genres, and timeframes to make sure that the data is diverse.**

In [5]:
df.isnull().sum()

Track                 0
Artist                0
Genre               813
Album Title           0
Album Type            0
Release Date          0
Thumbnail             0
Acousticness          0
Danceability          0
Energy                0
Instrumentalness      0
Popularity            0
Speechiness           0
Track ID              0
dtype: int64

In [6]:
df['Track ID'].duplicated().sum()

3188

It looks like there are a total or 813 null values in the genre column and a total of 3188 duplicate values in the 'Track ID' column.  The reason I chose to only look at the Track ID is because that is unique to the song, any repetition in that column means that the song was extracted multiple times.

Below I will drop all the duplicate Track ID data and the null values.

In [7]:
#Since 'cleaned' is taking a copy of the original dataframe I will use the copy() command to avoid any future warnings about using a slice of the data
cleaned = df.drop_duplicates(subset = 'Track ID', keep ='first').copy()
cleaned['Track ID'].duplicated().sum()

0

In [8]:
cleaned.dropna(subset = ['Genre'], inplace = True)

In [9]:
cleaned.isnull().sum()

Track               0
Artist              0
Genre               0
Album Title         0
Album Type          0
Release Date        0
Thumbnail           0
Acousticness        0
Danceability        0
Energy              0
Instrumentalness    0
Popularity          0
Speechiness         0
Track ID            0
dtype: int64

**Below I will look at the remaining duplicates but now I am focusing solely on the Track column, there are songs with the same title but different artist for example 'Work' by GangStarr and 'Work' by ASAP Ferg. In order to make sure I don't drop any duplicate song title by a different artist I will drop the songs based on both the Track and Artist column leaving me with the first instance of the song.**

In [10]:
#Like I mentioned above, these are likely duplicated songs with the same song title and different artist.
#To be sure I will drop the duplicates using the Track as well as the Artist column as a subset
cleaned['Track'].duplicated().sum()

858

In [11]:
cleaned.drop_duplicates(subset = ['Track', 'Artist'], keep = 'first', inplace = True)
cleaned.shape

(8716, 14)

In [12]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8716 entries, 0 to 13128
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track             8716 non-null   object 
 1   Artist            8716 non-null   object 
 2   Genre             8716 non-null   object 
 3   Album Title       8716 non-null   object 
 4   Album Type        8716 non-null   object 
 5   Release Date      8716 non-null   object 
 6   Thumbnail         8716 non-null   object 
 7   Acousticness      8716 non-null   float64
 8   Danceability      8716 non-null   float64
 9   Energy            8716 non-null   float64
 10  Instrumentalness  8716 non-null   float64
 11  Popularity        8716 non-null   int64  
 12  Speechiness       8716 non-null   float64
 13  Track ID          8716 non-null   object 
dtypes: float64(5), int64(1), object(8)
memory usage: 1021.4+ KB


---
Now that I have dealt with the duplicate data and the null values I will work on cleaning up the text and chaging some of the data types.

In [13]:
#I will convert the Release Date column to DateTime type. I need to do this to extract the release year for the songs later.
cleaned['Release Date'] = pd.to_datetime(cleaned['Release Date'])

In [14]:
cleaned['Release Year'] = cleaned['Release Date'].dt.year
cleaned.head()

,Track,Artist,Genre,Album Title,Album Type,Release Date,Thumbnail,Acousticness,Danceability,Energy,Instrumentalness,Popularity,Speechiness,Track ID,Release Year
0,"Shook Ones, Pt. II",Mobb Deep,"east coast hip hop, hardcore hip hop, hip hop,...",The Infamous,album,1995-04-25,https://i.scdn.co/image/ab67616d00004851a2203f...,0.01460,0.763,0.786,0.011400,78,0.229,33ZXjLCpiINn8eQIDYEPTD,1995
1,Hypnotize - 2014 Remaster,The Notorious B.I.G.,"east coast hip hop, gangster rap, hardcore hip...",Life After Death (2014 Remastered Edition),album,1997-03-04,https://i.scdn.co/image/ab67616d00004851fde79b...,0.14500,0.901,0.697,0.000000,83,0.280,7KwZNVEaqikRSBSpyhXK2j,1997
2,Ambitionz Az A Ridah,2Pac,"g funk, gangster rap, hip hop, rap, west coast...",All Eyez On Me,album,1996-02-13,https://i.scdn.co/image/ab67616d00004851a786ad...,0.00253,0.801,0.908,0.000655,2,0.221,3ssX20QT5c3nA9wk78V1LQ,1996
3,N.Y. State of Mind,Nas,"conscious hip hop, east coast hip hop, gangste...",Illmatic,album,1994-04-19,https://i.scdn.co/image/ab67616d00004851045fc9...,0.03940,0.665,0.910,0.000000,74,0.223,0trHOzAhNpGCsGBEu7dOJo,1994
4,It Was A Good Day,Ice Cube,"conscious hip hop, g funk, gangster rap, hip h...",The Predator,album,1992-11-17,https://i.scdn.co/image/ab67616d00004851994c31...,0.33000,0.798,0.744,0.000106,84,0.136,2qOm7ukLyHUXWyR4ZWLwxA,1992


Next, I will be normalizing the numeric columns. 

NOTE: Of all the numeric columns, the only one that needed to be normalized is Popularity. After looking at the Spotify API Documentation, Popularity is the only column that ranged from 1-100. For uniformity reasons I chose to keep it between 0 and 1 so that all numeric columns match.

In [15]:
cleaned['Popularity'] = (cleaned['Popularity'] - cleaned['Popularity'].min()) / (cleaned['Popularity'].max() - cleaned['Popularity'].min())
cleaned['Popularity'].head()

0    0.787879
1    0.838384
2    0.020202
3    0.747475
4    0.848485
Name: Popularity, dtype: float64

*Below I will save the cleaned dataframe as a new csv file that contains the clean data that will be used for EDA and modeling.*

Additionally I would like to mention that there will be some text cleaning which will be done during the modeling phase of the project. This is because I will be using Regex to create an additional column that will help ignore special characters in song titles.

In [17]:
cleaned.to_csv('../data/clean_data.csv')